In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import pandas as pd
import numpy as np
import os

In [1]:
# dimensions of our images.
img_width, img_height = 224, 224

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir ='../dogdata/training/'
validation_data_dir ='../dogdata/validation/'
nb_train_samples = 8239
nb_validation_samples = 1983
epochs = 30
batch_size = 10




In [3]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


In [4]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    #Obtaining Training Labels
    dirname='../dogdata/training'
    folders=os.listdir(dirname)
    label=[]
    for folder in folders:
        folder_path=os.path.join(dirname,folder)
        for img in os.listdir(folder_path):
            label+=[str(folder)]
    train_labels=pd.get_dummies(pd.DataFrame(label)).values[:8230,:]

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    dirname='../dogdata/validation'
    folders=os.listdir(dirname)
    label=[]
    for folder in folders:
        folder_path=os.path.join(dirname,folder)
        for img in os.listdir(folder_path):
            label+=[str(folder)]
    validation_labels=pd.get_dummies(pd.DataFrame(label)).values[:1980,:]

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [ ]:
test_data_dir ='../dogdata/test/'
datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode=None,
        shuffle=False)
from keras.models import load_model
model = load_model(top_model_weights_path)
preds = model.predict(test_generator, batch_size=1, verbose=1)
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)